# Reading retinal data with phy

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Enable PyQt4 event loop integration in IPython
from vispy.app import use_app
use_app('pyqt4')
%gui qt

In [3]:
import phy
from phy.io.h5 import open_h5
import phy.plot

In [4]:
%ls test-circus/

checkerboard.amplitudes-merged.mat*  checkerboard.spiketimes-merged.mat*
checkerboard.basis.npz*              checkerboard.templates-merged.mat*
checkerboard.clusters-merged.mat*    mea_252.mapping.mat*


## Reading the data

### Electrodes

In [5]:
fn_elec = 'test-circus/mea_252.mapping.mat'
f_elec = open_h5(fn_elec)
f_elec.describe()

/Mapping                                          (16, 16)            int16   
/Positions                                        (2, 252)            uint8   
/__globals__                                      (2,)                uint64  
/__header__                                       (75, 1)             uint16  
/__version__                                      (3, 1)              uint16  


In [6]:
mapping = f_elec.read('/Mapping')[...]
positions = f_elec.read('/Positions')[...].T.astype(np.float32)

In [7]:
f_elec.close()

### Templates

In [8]:
fn_temp = 'test-circus/checkerboard.templates-merged.mat'
f_temp = open_h5(fn_temp)
f_temp.describe()

/templates                                        (1116, 61, 252)     float32 


In [9]:
templates = f_temp.read('/templates')[...]

In [10]:
templates.shape  # this is equivalent to our 'waveforms' array: (n_templates, n_samples, n_channels)

(1116, 61, 252)

In [11]:
f_temp.close()

In [12]:
n_templates, n_samples, n_elec = templates.shape

### Spiketimes

In [13]:
fn_s = 'test-circus/checkerboard.spiketimes-merged.mat'
f_s = open_h5(fn_s)
#f_s.describe()

In [14]:
spiketimes = {i: f_s.read('temp_{}'.format(i))[:].ravel() for i in range(n_templates)
              if 'temp_{}'.format(i) in f_s.children()}

In [15]:
len(spiketimes)

588

In [16]:
f_s.close()

In [17]:
spiketimes[0]

array([   1857,    5419,   13774, ..., 4325002, 4325069, 4325420], dtype=int32)

### Clusters

In [18]:
fn_c = 'test-circus/checkerboard.clusters-merged.mat'
f_c = open_h5(fn_c)
#f_c.describe()

In [19]:
children = f_c.children()

shape = (n_spikes,) for each electrode

In [20]:
clusters = {i: f_c.read('/clusters_{}'.format(i))[:].ravel() for i in range(n_elec)
            if 'clusters_{}'.format(i) in children}

shape = (n_spikes, n_features) for each electrode

In [21]:
features = {i: f_c.read('/data_{}'.format(i))[...].T for i in range(n_elec)
            if 'data_{}'.format(i) in children}

In [22]:
f_c.close()

In [23]:
len(clusters), len(features)

(252, 252)

In [24]:
clusters[0].shape

(625,)

In [25]:
features[0].shape

(625, 140)

### Amplitudes

In [26]:
fn_a = 'test-circus/checkerboard.amplitudes-merged.mat'
f_a = open_h5(fn_a)
#f_a.describe()

In [27]:
children = f_a.children()

In [28]:
amplitudes = {i: f_a.read('temp_{}'.format(i))[...][0] for i in range(n_templates)
              if 'temp_{}'.format(i) in children}

In [29]:
amplitudes[0]

array([ 0.97597289,  0.73035991,  0.59705037, ...,  1.20012832,
        0.77556044,  0.74537909], dtype=float32)

In [30]:
len(amplitudes)

588

In [31]:
f_a.close()

List of variables:

* n_elec
* n_samples
* n_templates
* amplitudes
* clusters
* features
* mapping
* positions
* spiketimes
* templates

## Views

### Templates

In [32]:
templates.shape

(1116, 61, 252)

In [36]:
c = phy.plot.waveforms.plot_waveforms(templates[:1], channel_positions=positions)

Here is a slider allowing to show a given template:

In [37]:
from IPython.html.widgets import interact

In [38]:
@interact
def func(i=(0, n_templates)):
    # Update the waveforms: need to be a (n_spikes, n_samples, n_channels) array
    c.set_data(waveforms=templates[i])

TODO: take the mapping into account.

### Amplitudes

In [39]:
i = 0

In [40]:
f = amplitudes[i]
f /= f.max(axis=0)
f = -1. + 2 * f
f = f.reshape((-1, 1))

In [41]:
c = phy.plot.features.plot_features(f, spike_samples=spiketimes[i])

In [42]:
@interact
def func(i=(0, n_templates)):
    if i not in amplitudes:
        return
    f = amplitudes[i]
    f /= f.max(axis=0)
    f = -1. + 2 * f
    f = f.reshape((-1, 1, 1))
    
    s = spiketimes[i]
    
    c.set_data(features=f, spike_samples=s)

### Features

In [43]:
i = 0

In [44]:
f = features[i]
sc = clusters[i]

In [45]:
c = phy.plot.features.plot_features(f,
                                    spike_clusters=sc,
                                    # list of (channel, feature) tuple -- here, feature=0 always
                                    dimensions=[(0,0), (1,0), (2,0)])

In [46]:
@interact
def func(i=(0, n_elec)):
    if i not in clusters:
        return
    f = features[i]
    sc = clusters[i]
    c.set_data(f, spike_clusters=sc)